In [ ]:
%load_ext autoreload
%autoreload 2
# !pip install -e ../  # If not done yet...

In [ ]:
import os
import matplotlib.pyplot as plt

from pubrecon.data import DataFrame, ImagesData
from pubrecon.model import RCNN
from pubrecon.hyper import Hyper

In [ ]:
# General
data_path = "../data/in/"  # Where is your input data
work_path = "../data/out/"  # Where everything will be saved
seed = 1337  # Random seed
verbose = 1  # 0: no output; 1: normal informations; 2: e v e r y th i n g

# DataFrame
dataframe_pickle_path = os.path.join(work_path, "dataframe.pickle")  # Where will the DataFrame be saved
force_preparation = True  # Do you want to bypassed the saved DataFrame
subsamples = 128  # Number of samples to use for the DataFrame; -1: Use all of them

# ImagesData
imagesdata_pickle_path = os.path.join(work_path, 'imagesdata.pickle')  # Where will the ImagesData be saved
number_of_results = 2500  # How many samples will selective search use
iou_threshold = 0.85  # What is the percent of precision required
max_samples = 15  # How many class samples do you want
show_infos = True  # Show information for images output
show_labels = True  # Show labels for images output

# RCNN
model_and_weights_path = "../data/out/"  # Where will the model and weights be saved/loaded
loss = None  # Loss function; None: Use 
opt = None  # Optimization function; None: Use Adame
lr = 0.001  # Learning rate
epochs = 5  # Number of epochs
batch_size = 16
split_size = 0.15  # Test/Train proportion
checkpoint_path = os.path.join(work_path, 'checkpoint.h5')  # Where will the checkpoints be saved; None: No checkpoint (don't.)
early_stopping = True  # Should the learning stop if no more improvment is done
threshold = 0.85  # Threshold used for the recognition

In [ ]:
dataframe = DataFrame(data_path, pickle_path=dataframe_pickle_path)

In [ ]:
dataframe.prepare_data(force_preparation=force_preparation, subsamples=subsamples, verbose=verbose)

In [ ]:
dataframe.summary()

In [ ]:
imagesdata = ImagesData(dataframe, pickle_path=imagesdata_pickle_path)

In [ ]:
# That part is quite long, beware!
imagesdata.prepare_images_and_labels(number_of_results=number_of_results, iou_threshold=iou_threshold,
                                     max_samples=max_samples, verbose=verbose)

In [ ]:
x = imagesdata.images
y = imagesdata.labels

In [ ]:

np.save("x", x)
np.save("y", y)

In [1]:
import numpy as np

In [2]:
x = np.load("x.npy")
y = np.load("y.npy")

In [36]:
def rcnn(x_train, y_train, x_val, y_val, param):
    vggmodel = VGG16(weights='imagenet', include_top=True)  # https://keras.io/applications/

    # Freeze first 15 layers
    for i, layers in enumerate(vggmodel.layers[:15]):
        layers.trainable = False

    # Add a {number of classes} unit softmax dense layer
    predictions = Dense(len(set(y_train)), activation="softmax")(
        vggmodel.layers[-2].output)  # Maybe not all labels
    model = Model(input=vggmodel.input, output=predictions)

    # Compile the model using Adam optimizer with learning rate of 0.001 by default
    # We are using categorical_crossentropy as loss by default since the output of the model is categorical
    model.compile(loss=param['loss'], optimizer=param['opt'](lr=lr_normalizer(param['lr'], param['opt'])), metrics=["acc"])
    
    class MyLabelBinarizer(LabelBinarizer):
        def transform(self, y):
            Y = super().transform(y)
            if self.y_type_ == 'binary':
                return np.hstack((Y, 1 - Y))
            else:
                return Y

        def inverse_transform(self, Y, threshold=None):
            if self.y_type_ == 'binary':
                return super().inverse_transform(Y[:, 0], threshold)
            else:
                return super().inverse_transform(Y, threshold)

    chosen_binarizer = MyLabelBinarizer()
    train_labels_fit = chosen_binarizer.fit_transform(y_train)
    classes = chosen_binarizer.classes_

    # Dataset augmentation
    # This may not be needed following some magazines, as we do not often have rotated texts...
    # ... Or do we? Anyway it applies for the pictures so there's that.
    imgdatagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
    train_data = imgdatagen.flow(x=x_train, y=y_train, batch_size=param['batch_size'])
    imgdatagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
    test_data = imgdatagen.flow(x=x_val, y=y_val, batch_size=param['batch_size'])
    
    print(train_datata)

    # FINALLY train the model. https://keras.io/models/sequential/#fit_generator
    steps = ceil(len(train_data) / param['batch_size'])
    hist = model.fit(x_train, y_train, batch_size=param['batch_size'], steps_per_epoch=steps, epochs=param['epochs'], verbose=0,
                                         validation_data=[x_val, y_val], validation_steps=steps)
    
    return hist, model

In [37]:
p = {'loss': ['categorical_crossentropy'],
          'lr': (0.1, 10, 10),
          'epochs': [200],
          'opt': [Adam, Nadam],
          'batch_size': [2, 3, 4],
          'split_size': [0.1, 0.15],
         }

In [38]:
t = talos.Scan(x=x, y=y, params=p, model=rcnn, experiment_name='diabetes')










  0%|          | 0/120 [00:00<?, ?it/s]

ValueError: Error when checking target: expected dense_10 to have shape (25,) but got array with shape (1,)

In [4]:
from keras.optimizers import Adam, Nadam

Using TensorFlow backend.


In [6]:
import talos

In [7]:
from keras.applications.vgg16 import VGG16

In [8]:
from keras.layers import Dense

In [9]:
from keras import Model

In [10]:
from talos.utils import lr_normalizer

In [11]:
from sklearn.preprocessing import LabelBinarizer

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [13]:
from keras.losses import categorical_crossentropy

In [15]:
from math import ceil

In [ ]:
analyze_object = talos.Analyze(t)

In [ ]:
# get the highest result for any metric
analyze_object.high('val_acc')

In [ ]:
# get the round with the best result
analyze_object.rounds2high('val_acc')

In [ ]:
# get the best paramaters
analyze_object.best_params('val_acc', ['acc', 'loss', 'val_loss'])

In [ ]:


# heatmap correlation
analyze_object.plot_corr('val_loss', ['acc', 'loss', 'val_loss'])

In [ ]:
def model(x_train, y_train, x_val, y_val, params):
    vggmodel = VGG16(weights='imagenet', include_top=True)  # https://keras.io/applications/

    # Freeze first 15 layers
    for i, layers in enumerate(vggmodel.layers[:15]):
        layers.trainable = False

    # Add a {number of classes} unit softmax dense layer
    predictions = Dense(imagesdata.get_num_classes(), activation="softmax")(
        vggmodel.layers[-2].output)  # Maybe not all labels
    self.model = Model(input=vggmodel.input, output=predictions)

    # Compile the model using Adam optimizer with learning rate of 0.001 by default
    # We are using categorical_crossentropy as loss by default since the output of the model is categorical
    self.model.compile(loss=param['loss'], optimizer=param['opt'], metrics=["acc"])
    
    class MyLabelBinarizer(LabelBinarizer):
        def transform(self, y):
            Y = super().transform(y)
            if self.y_type_ == 'binary':
                return np.hstack((Y, 1 - Y))
            else:
                return Y

        def inverse_transform(self, Y, threshold=None):
            if self.y_type_ == 'binary':
                return super().inverse_transform(Y[:, 0], threshold)
            else:
                return super().inverse_transform(Y, threshold)

    chosen_binarizer = MyLabelBinarizer()
    train_labels_fit = chosen_binarizer.fit_transform(labels)
    self.classes = chosen_binarizer.classes_

    # Dataset augmentation
    # This may not be needed following some magazines, as we do not often have rotated texts...
    # ... Or do we? Anyway it applies for the pictures so there's that.
    imgdatagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
    train_data = imgdatagen.flow(x=x_train, y=y_train, batch_size=batch_size)
    imgdatagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=True, rotation_range=90)
    test_data = imgdatagen.flow(x=x_test, y=y_test, batch_size=batch_size)

    # FINALLY train the model. https://keras.io/models/sequential/#fit_generator
    steps = ceil(len(train_data) / param['batch_size'])
    self.hist = self.model.fit_generator(generator=train_data, steps_per_epoch=steps, epochs=param['epochs'], verbose=0,
                                         validation_data=test_data, validation_steps=steps)

In [ ]:
import talos

In [ ]:
scan_object = talos.Scan(images,
                         labels, 
                         model=model,
                         params=params,
                         experiment_name='aaa',
                         fraction_limit=.001)